In [1]:
from etr_case_generator.generator import ETRCaseGenerator
from etr_case_generator.reasoning_problem import ReasoningProblem
from etr_case_generator.ontology import ELEMENTS

from pyetr import View, State, SetOfStates
from pyetr.inference import default_inference_procedure, default_procedure_does_it_follow

In [ ]:
View.from_str("∃x {P(x)Q(x*)}")

<View
  stage={{<PredicateAtom predicate=<Predicate name=P arity=1> terms=(<ArbitraryObject name=x>)>,<PredicateAtom predicate=<Predicate name=Q arity=1> terms=(<ArbitraryObject name=x>)>}}
  supposition={{}}
  dep_rel=<DependencyRelation deps=[] unis=frozenset() exis=frozenset({x})>
  issue_structure={(<ArbitraryObject name=x>,<OpenPredicateAtom predicate=<Predicate name=Q arity=1> terms=(<QuestionMark>)>)}
  weights=<Weights {<PredicateAtom predicate=<Predicate name=P arity=1> terms=(<ArbitraryObject name=x>)>,<PredicateAtom predicate=<Predicate name=Q arity=1> terms=(<ArbitraryObject name=x>)>}: <Weight multi=<Multiset items=[]> add=<Multiset items=[]>>>
>


In [195]:
default_inference_procedure([
    View.from_str("Ex {S(x*)D(x)}"),
    View.from_str("{S(T()*)}"),
])

{D(T())}

In [ ]:
# ∀x { S(x) }^{ A(x) }
# { B(V) A(E) }
# { S(E) }

r = ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))
r.update_premises(premises=[
    View.from_str("Ex { S(x*) D(x) }"),
    View.from_str("{ S(T()*) }"),
])
r.update_query(query=View.from_str("{D(T())}"))

In [200]:
r.to_dict()

{'full_prose': "Consider the following premises:\n\n1. There is some X such that X is solid in vacuum and X is crystal-forming.\n2. Eclipsium is crystal-forming.\n\nDoes it follow that eclipsium is solid in vacuum?\n\nAnswer using 'YES' or 'NO' ONLY.",
 'premises': [('∃x {D(x)S(x*)}',
   'there is some X such that X is solid in vacuum and X is crystal-forming'),
  ('{S(T()*)}', 'eclipsium is crystal-forming')],
 'etr_conclusion': ('{D(T())}', 'eclipsium is solid in vacuum'),
 'etr_conclusion_is_categorical': True,
 'etr_conclusion_is_logically_consistent': False,
 'query': ('{D(T()*)}', 'eclipsium is solid in vacuum'),
 'etr_predicts_query_follows': True,
 'query_is_logically_consistent': False,
 'vocab_size': 4}

In [4]:
default_procedure_does_it_follow([View.from_str("Ax {C(x)}")], View.from_str("{C(B())}"))

False

In [5]:
print(r.full_prose())

Consider the following premises:

1. X is superconductive.

Does it follow that harmonium is superconductive?

Answer using 'YES' or 'NO' ONLY.


In [6]:
r.etr_conclusion

({0}, '')

In [7]:
view_queue = [View.get_falsum()]

In [8]:
from pyetr import View, State, SetOfStates

from etr_case_generator.mutate_view import random_existential_quantify, random_universal_quantify

v = View.from_str("{P(a())Q(b())R(c())}")
v = random_universal_quantify(v)
v = random_existential_quantify(v)
v = random_universal_quantify(v)
v

{R(c)P(a)Q(l)}
{c, l, a}
U={l} E={}
U={l} E={}
{P(a)R(i)Q(l)}
{i, l, a}
U={i,l} E={q}
U={i,l} E={}


∀i ∀l {P(a())R(i)Q(l)}

In [9]:
from pyetr import DependencyRelation, ArbitraryObject

DependencyRelation(
    universals=[ArbitraryObject(name="x")],
    existentials=[],
    dependencies=[],
).fusion(
    DependencyRelation(
    universals=[],
    existentials=[ArbitraryObject(name="y")],
    dependencies=[],
))

U={x} E={y}

In [10]:
import random

from etr_case_generator.generator import ETRCaseGenerator
from etr_case_generator.ontology import ELEMENTS
from etr_case_generator.reasoning_problem import ReasoningProblem
from etr_case_generator.mutate_view import mutate_view

r = ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))
r.update_premises(premises=[View.get_falsum()])

r.premises

[({}, '')]

In [11]:
def mutate_reasoning_problem(r: ReasoningProblem, max_premises: int = 3) -> ReasoningProblem:
    if len(r.premises) > 0 and random.random() < 0.5:
        # Mutate one of the existing premises
        i = random.randint(0, len(r.premises) - 1)
        premises = [p[0] for p in r.premises]
        premises[i] = mutate_view(premises[i])
        r.update_premises(premises=premises)
        return r
    elif len(r.premises) <= max_premises and random.random() < 0.5:
        # Add a new premise
        r.update_premises(premises=[p[0] for p in r.premises] + [View.get_falsum()])
        return r
    else:
        # Mutate query
        r.update_query(query=mutate_view(r.query[0]))
        return r

def ignore_problem(r: ReasoningProblem) -> bool:
    if len(r.premises) == 0:
        return True
    return len(r.premises) <= 1 and r.premises[0][0].is_falsum

problem_queue = [ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))]    

In [12]:
View.from_str("Ax Ey {P(x, y)}").dependency_relation
# dependency captures the existentials that depend on universals
# This means that otherwise noted, the existentials are scoped outside of the universals

U={x} E={y} deps=x{y}

In [13]:
View.from_str("Ex Ay {P(x, y)}").to_str()  # we can just cheat and use the to_str order

'∃x ∀y {P(x,y)}'

In [14]:
import re
delims = r"[ ∃∀]"
re.split(delims, '∃x ∀y {P(x,y)}'.split("{")[0])

['', 'x', '', 'y', '']

In [17]:
import time
while len(problem_queue) < 100:
    r = problem_queue.pop(0)
    if not ignore_problem(r): problem_queue.append(r)
    problem_queue.append(mutate_reasoning_problem(r))

    if not any([p[0].is_falsum for p in r.premises]):
        print(r.premises)
        print(r.query)
        print(r.full_prose())
    # time.sleep(1)

ValueError: Universals with existentials: frozenset() not the same as those in stage/supposition {y}

In [ ]:
from etr_case_generator.generator import ETRCaseGenerator
from etr_case_generator.ontology import ELEMENTS
from etr_case_generator.view_to_natural_language import view_to_natural_language

g = ETRCaseGenerator(ELEMENTS)

view_to_natural_language(view_queue[-1])

TypeError: view_to_natural_language() missing 1 required positional argument: 'view'

In [ ]:
view_to_natural_language(g.add_quantification(view_queue[0]))

AttributeError: 'ETRCaseGenerator' object has no attribute 'add_quantification'

[{}]

In [ ]:
m.merge_random_unary_predicate()

In [ ]:
m.view

{S(L())B()I(B()),A()S(L())I(B())}^{C()}

In [ ]:
[a.predicate.name for a in m.view.atoms]

['A', 'C', 'B']

In [ ]:
m.factor_random_atom()
m.view

{B()}^{C()}

In [ ]:
m.factor_random_atom()
m.view

{C()}^{}

In [ ]:
st = list(View.from_str("{A()B()}").stage)[0]
from pyetr.view import state_division
state_division(st, )

AttributeError: 'State' object has no attribute 'division'

In [ ]:
ab = View.from_str("{A(), B()}")
a = View.from_str("{A()}")
b = View.from_str("{B()}")
c = View.from_str("{C()}")

In [ ]:
res = View.from_str("{p()q()r(), q()s(), p()}").factor(View.from_str("{p(), q()}"))
res

{s(),0,q()r()p()}

In [ ]:
res.answer(View.get_verum())

{s(),0,q()r()p()}

In [ ]:
res.answer(res)

{q()r()p()}

In [ ]:
ab.product(View.get_verum())

{A(),B()}

In [ ]:
ans = ab.factor(a)

In [ ]:
ans.answer(ans)

{B()}

In [ ]:
a = list(v.atoms)[0]
a_v = View.with_defaults(stage=SetOfStates({State({a})}))

In [ ]:
a_v

{A()}

In [ ]:
v.factor(a_v) #.product(View.get_falsum())

{B(),0}

In [ ]:
v.division(View.from_str("{A()}")).factor(View.get_verum())

{0,B()}

In [ ]:
v.merge(View.from_str("{C()}"))

{C()B(),A()C()}

# Editing Views with Generator

In [ ]:
g = ETRCaseGenerator(ontology=ELEMENTS)

In [ ]:
view = View.from_str("{P(a()), Q(b())}^{R(a())}")

In [ ]:
# ret = g.add_quantification(view)

In [ ]:
a = list(view.atoms)[0]
t = a.terms[0]
t

b

In [ ]:
new_t = list(view.atoms)[2].terms[0]

In [ ]:
new_t

a

In [ ]:
g.replace_term_in_atom(a, term_to_replace=t, replacement_term=new_t)

Q(a)